# use the yang zhang paper (1104.2487v2) on weighting for magnification bias. We want the same forecasts as in the S2N notebook, but using the weighting function from menard and bartelmann

We need a lot of averages for this, all with respect to magnitude, in particular we need:

$\alpha-1$
$b_g$ and
$W$

$W$ itself requires the same averages.

In [11]:
%matplotlib inline

from __future__ import division
import numpy as np
from numpy import pi,sin,cos,tan,e,arctan,arcsin,arccos,sqrt
import matplotlib
import matplotlib.pyplot as P

from matplotlib.legend import Legend

matplotlib.rcParams['figure.figsize'] = (9, 6)
P.rcParams['text.usetex'] = True  # not really needed
P.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}'] #for \text command
P.rcParams["font.size"] = 26
P.rc('xtick', labelsize=23) 
P.rc('ytick', labelsize=23)
# P.rc('xtick', labelsize=20) 
# P.rc('ytick', labelsize=20)
np.set_printoptions(threshold=100000)


from magmod import *
from magbias_experiments import SKA1, SKA2, cb_hirax as hirax, LSST, n

#max magnitude for LSST...
MAXMAG = 27

bgal_new has the same magnitude dependence as sg, ONLY ON THE BIN CENTER

shotnoise for the magnitude bin is straight forward

C_DM does not depend on magnitude

In [12]:
def Ngal_in_bin(zmin, mtab_edges, galsurv, NINT = 2000, maxmag = MAXMAG, ZMAX = False):
    """mtab_edges is magnitude bin edges, 
    upper means higher number and fainter, thus more galaxies"""

    if type(ZMAX)==bool and not ZMAX:
        zmax = zbg_max(maxmag, galsurv)
    elif type(ZMAX)==bool and ZMAX:
        raise ValueError("ZMAX must either be False or float!")
    else:
        zmax = ZMAX
    z_integrate = np.linspace(zmin,zmax, NINT)


    dNdztab = np.array([nofz(z_integrate, mmm) for mmm in mtab_edges])
    dNdzdOm = np.trapz(dNdztab, z_integrate, axis = 1)
    

    
    #now subtract lower from upper
    Nz = (dNdzdOm[1:] - dNdzdOm[:-1]) * 4 * np.pi #already in rad!!!! and multiplying with all sky
    Nz = np.atleast_1d(Nz)
    if (Nz<0).any():
#         print Nz
        raise ValueError("cannot have negative number of galaxies, most likely we confused magnitudes here")
    return Nz





def alphaminusone(z, maxmag, exp = LSST): #2(alpha-1) = 5sg-2
    """should take the mean redshift in the bin!"""
    maxmag = np.atleast_1d(maxmag)
    res = np.array( [ (5*sg(z, exp, mmm) - 2)/2 for mmm in maxmag])
    return res

def average_no_W(A,B,N, VECTORINPUT = False):
    """A,B are the arrays to be averaged, same length as N which is the bg number density of galaxies"""
    if VECTORINPUT:
        sumaxis = 0
    else:
        sumaxis = 1
    return np.sum(A*B*N, axis = sumaxis)/np.sum(N) #A and B are matrices, ell x mag



def W_weight(C_DMtab, Cshottab, alpha_m_one_tab, bgal_tab, Ngal_tab, exp = LSST, MENARD_BARTELMANN = False):
    """the optimal, scale dependent weight, eq. 10
    input: background redshift z, magnitude bin center mag (later will be full array, now just needs to be inside the table used for bgal_tab),
    array (for ell values) of dark matter power spectrum and the shot noise (number). 
    Then there are tables of alpha-1, bgal, Cshot and Ngal on a given magnitude binning (the binning itself not needed here)
    if MENARD_BARTELMANN is True we use their weighting function, which ignores the galaxy bias term"""

    DM_by_shot = np.outer(C_DMtab,1/Cshottab) #this need to have right shape, as C_DM is length ell and Cshot is length magtab
    
    if MENARD_BARTELMANN:
        res = alpha_m_one_tab + np.zeros(DM_by_shot.shape)
        
#         print "Warning! weight is set to 1"
#         return np.ones(res.shape)

        return res
    
    num = -average_no_W(alpha_m_one_tab, bgal_tab, Ngal_tab, VECTORINPUT = True) * DM_by_shot
    denom = 1 + average_no_W(bgal_tab,bgal_tab, Ngal_tab, VECTORINPUT = True) * DM_by_shot
    fac1 = num/denom
    res = alpha_m_one_tab + bgal_tab * fac1
    
    return res #outer product to get right shape



def S2N_weighted_old(ltab, deltaell, fsky, Cfg, CSfg, CSbg, CHImu, biasg, Weight, Ngal, alphaminus1):
#     num = (2*ltab + 1) * deltaell * fsky
    num = (2*ltab + 1) * deltaell * fsky / 2 #divided by two to compare with our results
    try:
        denom = 1 + (Cfg + CSfg) * ( 
            average_no_W(biasg, Weight, Ngal)**2 * C_DM_tab + 
            average_no_W(Weight, Weight, Ngal) * CSbg) / (average_no_W(CHImu, Weight, Ngal)**2)
        frac = num/denom
    except:
        print Weight.shape, alphaminus1.shape, Ngal.shape, Ckappag.shape
        raise
    if (frac<0).any():
        print "We are having negative S2N, therefore we use the absolute values."
        raise
#         frac = np.abs(frac)
    res = np.sum(frac, axis = 0)
    return np.sqrt(res)


def S2N_weighted(ltab, deltaell, fsky, Cfg, CSfg, CSbg, CHIkappa, biasg, Weight, Ngal, alphaminus1):
    num = (2*ltab + 1) * deltaell * fsky
    try:
        denom = 1 + (Cfg + CSfg) * ( 
            average_no_W(biasg, Weight, Ngal)**2 * C_DM_tab + 
            average_no_W(Weight, Weight, Ngal) * CSbg) / (4*average_no_W(Weight, alphaminus1, Ngal)**2 * CHIkappa**2)
        frac = num/denom
    except:
        print Weight.shape, alphaminus1.shape, Ngal.shape, CHIkappa.shape
        raise
    if (frac<0).any():
        print "We are having negative S2N, therefore we use the absolute values."
        raise
#         frac = np.abs(frac)
    res = np.sum(frac, axis = 0)
    return np.sqrt(res)

def S2N_weighted_new(ltab, deltaell, fsky, Cfg, CSfg, CSbg, CHIkappa, biasg, Weight, Ngal, alphaminus1):
    num = (2*ltab + 1) * deltaell * fsky
    try:
        denom = 1 + (Cfg + CSfg) * ( 
            average_no_W(biasg, Weight, Ngal)**2 * C_DM_tab + 
            average_no_W(Weight, Weight, Ngal) * CSbg) / (4*average_no_W(CHIkappa*Weight,alphaminus1, Ngal)**2)
        frac = num/denom
    except:
        print Weight.shape, alphaminus1.shape, Ngal.shape, CHIkappa.shape
        raise
    if (frac<0).any():
        print "We are having negative S2N, therefore we use the absolute values."
        raise
#         frac = np.abs(frac)
    res = np.sum(frac, axis = 0)
    return np.sqrt(res)

setting up experiments and plotting like in the S2N notebook:

In [13]:
lw = 2.5 #line width forplots

buffer_z = 0.1 #buffer between fg and bg to avoid correlation, 
zfg_max = 2. # last bin goes slightly beyond that. more does not make sense otherwise we have no galaxies left to cross correlate with

In [14]:
#get lmin = delta_l for HIRAX and ska:

# allskydegree = 4*pi*(180/pi)**2

SKAarea = SKA['S_area']
HIRAXarea = hirax['S_area']
HIRAXfov1 = 15 * (pi/180)**2 #this is from table one of the HIRAX white paper
HIRAXfov2 = 56 * (pi/180)**2
HIRAXfov0 = (HIRAXfov1+HIRAXfov2)/2

print "HIRAX fov = {} to {} rad".format(HIRAXfov1, HIRAXfov2)

lminSKA = np.amax([10,np.int(np.around(2*pi/np.sqrt(SKAarea)))]) #never use lmin <20
lminHIRAX = np.amax([10, np.int(np.around(2*pi/np.sqrt(HIRAXfov0)))]) #never use lmin<20
print "min ell SKA: {}, min ell HIRAX: {}".format(lminSKA,lminHIRAX)


#less ell resolution for speedup:
deltaell_HIRAX=10*lminHIRAX
deltaell_SKA=10*lminSKA
print "we are using rough ell res for speedup"

# deltaell_HIRAX = lminHIRAX
# deltaell_SKA = lminSKA

HIRAX fov = 0.0045692612968 to 0.0170585755081 rad
min ell SKA: 10, min ell HIRAX: 60
we are using rough ell res for speedup


In [15]:
#set the magnitude range:
magmax = 27;
magmin = 19;
# magmin = 22;

#make ell table:
lend = 2200;
ltabH = np.arange(lminHIRAX, lend + deltaell_HIRAX, deltaell_HIRAX)
ltabSKA = np.arange(lminSKA, lend + deltaell_SKA, deltaell_SKA)

# lN = len(ltab)
lNH = len(ltabH)
lNSKA = len(ltabSKA)

HIRAX redshift bins:

In [16]:

########################################################################
########################################################################
############WORKING ON HIRAX:####################################

dz = 0.5 #bin width for HIRAX and SKA


########################################################################
########################################################################
############MAKING HIRAX REDSHIFT BINS:####################################
########################################################################
########################################################################

numax_HIRAX = hirax["survey_numax"] #800

zHIRAX_min = nutoz21(numax_HIRAX) # minimum of HIRAX

print zfg_max, zHIRAX_min, "zmax, zmin for HIRAX"
zHIRAX_allbin = np.arange( zHIRAX_min, zfg_max + dz, step = dz)
print zHIRAX_allbin, "all"
zHIRAX_lowbin = zHIRAX_allbin[:-1]
zHIRAX_highbin = zHIRAX_allbin[1:]
zHIRAX_meanbin = (zHIRAX_lowbin + zHIRAX_highbin)/2


zbmin_HIRAX = zHIRAX_highbin + buffer_z

nbinH = len(zHIRAX_lowbin)
print zHIRAX_lowbin, "low"
print zHIRAX_highbin, "high"
print zHIRAX_meanbin, "mean"
print zbmin_HIRAX, "bg min"
print nbinH, "Nbin"




2.0 0.7755075 zmax, zmin for HIRAX
[0.7755075 1.2755075 1.7755075 2.2755075] all
[0.7755075 1.2755075 1.7755075] low
[1.2755075 1.7755075 2.2755075] high
[1.0255075 1.5255075 2.0255075] mean
[1.3755075 1.8755075 2.3755075] bg min
3 Nbin


SKA redshift bins:

In [17]:

# Band1:
zSKA1_min = np.amax([0.0005, nutoz21(SKA1["numax"])]) # minimum of HIRAX

zSKA1_max = np.amin([zfg_max, nutoz21(SKA1["numin"])]) # last bin goes slightly beyond that. more does not make sense otherwise we have no galaxies left to cross correlate with

print "Band 1:"
print "zmax {} and zmin {}".format(zSKA1_max, zSKA1_min)

zSKA1_allbin = np.arange( zSKA1_min, zSKA1_max + dz, step = dz)
print zSKA1_allbin, "all"
zSKA1_lowbin = zSKA1_allbin[:-1]
zSKA1_highbin = zSKA1_allbin[1:]
zSKA1_meanbin = (zSKA1_lowbin + zSKA1_highbin)/2

zbSKA1_min = zSKA1_highbin + buffer_z #background


nbinS1 = len(zSKA1_lowbin)
print zSKA1_lowbin, "low"
print zSKA1_highbin, "high"
print zSKA1_meanbin, "mean"
print nbinS1, "BINS"


# Band2:
zSKA2_min = np.amax([0.0005, nutoz21(SKA2["numax"])]) # minimum of HIRAX

zSKA2_max = np.amin([zfg_max, nutoz21(SKA2["numin"])]) # last bin goes slightly beyond that. more does not make sense otherwise we have no galaxies left to cross correlate with

print "\nBand 2:"
print "zmax {} and zmin {}".format(zSKA2_max, zSKA2_min)

# zSKA2_allbin = np.arange( zSKA2_min, zSKA2_max + dz, step = dz)
dzSKA2 = zSKA2_max - zSKA2_min
zSKA2_allbin = np.array( [zSKA2_min, zSKA2_max])

print dzSKA2, "dz"
print zSKA2_allbin, "all"
zSKA2_lowbin = zSKA2_allbin[:-1]
zSKA2_highbin = zSKA2_allbin[1:]
zSKA2_meanbin = (zSKA2_lowbin + zSKA2_highbin)/2

zbSKA2_min = zSKA2_highbin + buffer_z #background


nbinS2 = len(zSKA2_lowbin)
print zSKA2_lowbin, "low"
print zSKA2_highbin, "high"
print zSKA2_meanbin, "mean"
print nbinS2, "BIN"



Band 1:
zmax 2.0 and zmin 0.341271010387
[0.34127101 0.84127101 1.34127101 1.84127101 2.34127101] all
[0.34127101 0.84127101 1.34127101 1.84127101] low
[0.84127101 1.34127101 1.84127101 2.34127101] high
[0.59127101 1.09127101 1.59127101 2.09127101] mean
4 BINS

Band 2:
zmax 0.470399585921 and zmin 0.0005
0.469899585921325 dz
[0.0005     0.47039959] all
[0.0005] low
[0.47039959] high
[0.23544979] mean
1 BIN


In [18]:
#
zbin_labels = ["H1", "H2", "H3", "SKA11", "SKA12", "SKA13", "SKA14", "SKA21"]
zbin_lowlist = [zzz for zzz in zHIRAX_lowbin] + [zzz for zzz in zSKA1_lowbin] + [zzz for zzz in zSKA2_lowbin]
zbin_highlist = [zzz for zzz in zHIRAX_highbin] + [zzz for zzz in zSKA1_highbin] + [zzz for zzz in zSKA2_highbin]

elltabs_list = [ltabH]*3 +  [ltabSKA]*5
d_ell_list = [deltaell_HIRAX]*3 + [deltaell_SKA]*5

fgexp_list = [hirax]*3 + [SKA1]*4 + [SKA2]


for i in range(len(zbin_labels)):
    print "Bin {} goes from z = {} to z = {}.".format(zbin_labels[i], zbin_lowlist[i], zbin_highlist[i])
    print "the ell table is {}".format(elltabs_list[i])
    print "ellmin = {}".format(d_ell_list[i])
    print "mode of fg exp is {} \n \n".format(fgexp_list[i]["mode"])

Bin H1 goes from z = 0.7755075 to z = 1.2755075.
the ell table is [  60  660 1260 1860 2460]
ellmin = 600
mode of fg exp is interferometer 
 

Bin H2 goes from z = 1.2755075 to z = 1.7755075.
the ell table is [  60  660 1260 1860 2460]
ellmin = 600
mode of fg exp is interferometer 
 

Bin H3 goes from z = 1.7755075 to z = 2.2755075.
the ell table is [  60  660 1260 1860 2460]
ellmin = 600
mode of fg exp is interferometer 
 

Bin SKA11 goes from z = 0.341271010387 to z = 0.841271010387.
the ell table is [  10  110  210  310  410  510  610  710  810  910 1010 1110 1210 1310
 1410 1510 1610 1710 1810 1910 2010 2110 2210]
ellmin = 100
mode of fg exp is single_dish 
 

Bin SKA12 goes from z = 0.841271010387 to z = 1.34127101039.
the ell table is [  10  110  210  310  410  510  610  710  810  910 1010 1110 1210 1310
 1410 1510 1610 1710 1810 1910 2010 2110 2210]
ellmin = 100
mode of fg exp is single_dish 
 

Bin SKA13 goes from z = 1.34127101039 to z = 1.84127101039.
the ell table is [  10  

In [19]:
#weighting function by menard and bartelmann:
menard_bartelmann = True

#set the number of galaxies to zero below a certain threshold. the threshold should correspond to the minimum number of galaxies for a detection.
N_g_threshold = 1

#subsplitting of LSST bg bin, later added up
dzb = 0.1
zmax_LSST = LSST["zmax"]

#number of magnitude bins:
Nmag = 30
# Nmag = 10

#magnitude bins:
magtab_edges = np.linspace(magmin, magmax, Nmag+1)
# print "magnitude bin edges: {}".format(magtab_edges)

#coarse integration because z bins are small in bg
NINTEGRATE = 5

#list for all S2N values in each z bin.
allS2N = []
#loop through all HI z bins:
for izfg in range(len(zbin_labels)):
# for izfg in range(1):
    print "\n"*2, "*"*25
    print "\n", "bin {}".format(zbin_labels[izfg])
    zflow = zbin_lowlist[izfg]; zfhigh = zbin_highlist[izfg]; zfmean = (zfhigh+zflow)/2; delta_zf = (zfhigh-zflow)/2
    print "z from {} to {}".format(zflow, zfhigh)

    #the ell tab used for this HI bin:
    elll = elltabs_list[izfg]
    delll = d_ell_list[izfg]
    
    
    #the fg experiment:
    fgexxp = fgexp_list[izfg]
    
    zlow_LSST = zfhigh + buffer_z
    #background LSST zbins
    zb_tab, zb_step = np.linspace(zlow_LSST, zmax_LSST, np.int((zmax_LSST-zlow_LSST)/dzb+1), retstep = True)
#     print zb_tab, zb_step

    #corresponding tab of alpha-1, zb_step added to get mean redshift. shape (mag X ell)
    alphaminusones = alphaminusone(zb_tab+zb_step/2, magtab_edges[1:]) 

    
    
    if fgexxp["mode"] == "single_dish":
        C_noise_fg_tab = noise_cls_single_dish(elll, ztonu21(zfmean), fgexxp, 256) * np.ones( len(elll) )

    elif fgexxp["mode"] == "interferometer":
        C_noise_fg_tab = Cl_interferom_noise(elll, zflow, zfhigh, fgexxp)

    else:
        raise ValueError("Foreground experiment needs to be either in single_dish or interferometer mode.")
    print "using {} mode for foreground".format(fgexxp["mode"])
    
    
    fsky = fgexxp["S_area"] / (4*np.pi)
    print "delta ell = {}, fsky = {}".format(delll, fsky)


    
    
    S2Ntab = []
    for iz in range(len(zb_tab)-1):#all but the last entry

        zlow = zb_tab[iz]
        zhigh = zb_tab[iz+1]
        zbmean = (zlow + zhigh)/2

        N_g_tab = Ngal_in_bin(zlow, magtab_edges, LSST, ZMAX = zhigh, NINT = NINTEGRATE) #takes the lower redshift edge
#         print bias_g_tab, "bg"
        if np.isnan(N_g_tab).any():
            print N_g_tab, "N_g with nan"
        N_g_tab[N_g_tab<N_g_threshold] = 0. #less than one galaxy is the same as no galaxy...

#         print N_g_tab, "Ngal"
        alphaminusone_tab = alphaminusones[:,iz]
        alphaminusone_tab_notsetto1 = alphaminusone_tab[:]
        
        alphaminusone_tab[N_g_tab<N_g_threshold] = 1.#exact value doesn't matter, it's multiplied with 0!
        
        bias_g_tab = np.array([bgal_new(zbmean, mmm) for mmm in magtab_edges[1:]])
        bias_g_tab[N_g_tab<N_g_threshold] = 1. #exact value doesn't matter, it's multiplied with 0!

#         print "alphaminusone", alphaminusone_tab

        C_DM_tab = C_l_DM_CAMB(elll, zlow, galsurv = LSST, ZMAX = zhigh)

        Cshot = shotnoise(zlow, LSST, MAXMAG, ZMAX = zhigh, NINT = NINTEGRATE) #coarse integration okay because we use thin redshift bin 



        Weight = W_weight(C_DM_tab, Cshot, alphaminusone_tab, bias_g_tab, N_g_tab, MENARD_BARTELMANN  = menard_bartelmann)
        if (Weight>1e10).any():
            print "ALARM!"
            print Weight[:,N_g_tab<N_g_threshold]
#         print Weight, "Weight"
        C_HIHI_tab = C_l_HIHI_CAMB(elll, zflow, zfhigh)
        

        C_HIxmag_tab = np.array( [Cl_HIxmag_CAMB(elll, zfmean, delta_zf, zlow, MAXMAG = mmm, ZMAX = zhigh, NINT_gkernel = NINTEGRATE)
                                  for mmm in magtab_edges[1:]]).T #transpose to match shape
#         C_HIkappa_tab = np.array( [Cl_kappa_HI_CAMB(elll, zfmean, delta_zf, zlow, MAXMAG = mmm, ZMAX = zhigh, NINT_gkernel = NINTEGRATE)
#                                   for mmm in [magtab_edges[0]]][0]).T #transpose to match shape
#         C_HIkappa_tab = np.array( [Cl_kappa_HI_CAMB(elll, zfmean, delta_zf, zlow, MAXMAG = mmm, ZMAX = zhigh, NINT_gkernel = NINTEGRATE)
#                                   for mmm in magtab_edges[1:]]).T #transpose to match shape
#         print "this should all be the same: {}".format([Cl_kappa_HI_CAMB(elll, zfmean, delta_zf, zlow, MAXMAG = mmm, ZMAX = zhigh, NINT_gkernel = NINTEGRATE)
#                                   for mmm in magtab_edges])
#         print "This should be about one: {}".format(C_HIxmag_tab[:,-1]/C_HIkappa_tab[:,-1]/2/alphaminusone_tab_notsetto1[-1])

#         S2Ntmp = S2N_weighted(elll, delll, fsky, C_HIHI_tab, C_noise_fg_tab, Cshot, C_HIkappa_tab, bias_g_tab, Weight, N_g_tab, alphaminusone_tab)
#         S2Ntmp = S2N_weighted_new(elll, delll, fsky, C_HIHI_tab, C_noise_fg_tab, Cshot, C_HIkappa_tab, bias_g_tab, Weight, N_g_tab, alphaminusone_tab)
        S2Ntmp = S2N_weighted_old(elll, delll, fsky, C_HIHI_tab, C_noise_fg_tab, Cshot, C_HIxmag_tab, bias_g_tab, Weight, N_g_tab, alphaminusone_tab)
        S2Ntab.append(S2Ntmp)
        if np.isnan(S2Ntmp):
            break #no more S2N to get
    S2Ntab = np.array(S2Ntab)
    #now squared summation over the background z bins:
    S2Nsqsum = np.sqrt(np.nansum(S2Ntab**2))
    allS2N.append( S2Nsqsum )
    print "S2N in this bin: {}".format(S2Nsqsum)




*************************

bin H1
z from 0.7755075 to 1.2755075
using interferometer mode for foreground
delta ell = 600, fsky = 0.3625


/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in true_divide
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:84: RuntimeWarning: invalid value encountered in less


S2N in this bin: 51.5542136877


*************************

bin H2
z from 1.2755075 to 1.7755075
using interferometer mode for foreground
delta ell = 600, fsky = 0.3625
S2N in this bin: 14.9599847934


*************************

bin H3
z from 1.7755075 to 2.2755075
using interferometer mode for foreground
delta ell = 600, fsky = 0.3625
S2N in this bin: 3.78237063476


*************************

bin SKA11
z from 0.341271010387 to 0.841271010387
using single_dish mode for foreground
delta ell = 100, fsky = 0.409823978462


/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:79: RuntimeWarning: overflow encountered in true_divide


S2N in this bin: 14.8207219959


*************************

bin SKA12
z from 0.841271010387 to 1.34127101039
using single_dish mode for foreground
delta ell = 100, fsky = 0.409823978462
S2N in this bin: 5.72653286107


*************************

bin SKA13
z from 1.34127101039 to 1.84127101039
using single_dish mode for foreground
delta ell = 100, fsky = 0.409823978462
S2N in this bin: 1.71462950685


*************************

bin SKA14
z from 1.84127101039 to 2.34127101039
using single_dish mode for foreground
delta ell = 100, fsky = 0.409823978462
S2N in this bin: 0.309213696918


*************************

bin SKA21
z from 0.0005 to 0.470399585921
using single_dish mode for foreground
delta ell = 100, fsky = 0.409823978462
S2N in this bin: 20.4698974131
